In [2]:
import pandas as pd
import snowflake.connector
from info import *
from snow import *
import numpy as np
import pandas as pd
#import pycaret
from pycaret.time_series import *


'USA','CAN','MEX','CRI','PAN','BRA','ARG','CHL','URY','BOL','PER','EGY','LBY','ZAF','NGA', 
    'MAR','AUS','CHN','IND','THA','JPN','KOR','ISR','SAU','MYS','USA','IDN','RUS','TUR','ESP','BGR',
    'FRA','ITA','DEU','GBR','NOR','SWE','GRC'

In [36]:
pais=['USA','CAN','MEX','CRI','PAN','BRA','ARG','CHL','URY','BOL','PER','EGY','LBY','ZAF','NGA', 
    'MAR','AUS','CHN','IND','THA','JPN','KOR','ISR','SAU','MYS','USA','IDN','RUS','TUR','ESP','BGR',
    'FRA','ITA','DEU','GBR','NOR','SWE','GRC']   
resultados = pd.DataFrame()

In [3]:
cnn = snowflake.connector.connect(

       user=snow_user,
        password=snow_password,
        account=snow_account,
        warehouse=snow_warehouse,
        database=snow_database)
sql ="""SELECT p.CODIGO_PAIS, e.ANIO, e.VALOR, i.DESCRIPCION as INDICADOR 
            FROM EV e 
            JOIN INDICADOR i 
            ON (e.ID_INDICADOR=i.ID_INDICADOR)
            JOIN PAIS p
            on (e.ID_PAIS=p.ID_PAIS)
            WHERE e.ID_INDICADOR=31 AND e.ANIO>1965 AND e.ANIO<=2020
             """ 
data=pd.read_sql(sql,cnn)
data3=data.copy()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [37]:
for i in pais:
    data2= pd.DataFrame()
    pais2=()
    pais2=(i)   
    data2=data3[(data3['CODIGO_PAIS'] ==pais2)]

    data2=data2.rename(columns={'VALOR':'EV_Total'})

    data2['date']=data2['ANIO']
    data2['date'] = pd.to_datetime(data2['date'],format='%Y')

    data2=data2.set_index('date')
    data2.drop(['CODIGO_PAIS','ANIO','INDICADOR'], axis = 1,inplace=True)

    
    exp = TSForecastingExperiment() 
    s = setup(data2,target='EV_Total', fh = 10, fold=3, session_id = 123, verbose=False,scale_target='robust') #, enforce_exogenous = False
            
    best =  s.compare_models(n_select=3, turbo=False, verbose=False,exclude=['omp_cds_dt','knn_cds_dt'] )   

    #tuned_model = [s.tune_model(best, choose_better=True)]
    #tuned_model = [s.tune_model(best, choose_better=True,n_iter=50)]#, verbose=False

    tuned_model = [s.tune_model(model, choose_better=True,n_iter=50, verbose=False) for model in best]#
    #tuned_model = [s.tune_model(model ,verbose=False) for model in best]
    #tuned_model1b = s1.tune_model(best1, search_algorithm = "grid" ,verbose=False)

    blender = s.blend_models(tuned_model, choose_better=True, verbose=False)#

    #blender = s.blend_models(tuned_model, method='voting', weights=top_model_weights.values.tolist(),verbose=False)

    final_best1 = s.finalize_model(blender)
    #final_best1b = s1.finalize_model(tuned_model1b)

    df_fut1=predict_model(final_best1, fh = 10)
    resultados[i] = df_fut1

In [32]:
resultados

,USA,CAN,MEX,CRI,PAN,BRA,ARG,CHL,URY,BOL,...,TUR,ESP,BGR,FRA,ITA,DEU,GBR,NOR,SWE,GRC
2021,77.4768,81.7313,75.2197,80.6508,78.8813,76.3372,76.9589,80.4879,78.2053,72.0152,...,78.3090,83.5965,73.6565,82.4261,84.5703,81.3048,81.4953,83.3469,83.4640,81.3777
2022,77.3891,81.7256,75.3159,80.8339,79.0385,76.5553,77.1049,80.6586,78.3600,72.2484,...,78.6836,83.1865,73.7113,82.6638,84.8225,81.3201,81.6105,83.5310,83.7039,81.6648
2023,77.1681,81.7198,75.4154,81.0116,79.1948,76.7724,77.2512,80.8414,78.5206,72.4731,...,79.0518,83.7898,73.7821,82.9014,85.0748,81.4247,81.7273,83.6971,83.8844,81.9492
2024,76.9470,81.7140,75.5141,81.1810,79.3498,76.9852,77.3978,81.0359,78.6867,72.6916,...,79.4136,83.7792,73.8528,83.1391,85.3271,81.6831,81.8405,83.8804,84.0564,82.2310
2025,76.7259,81.7082,75.6089,81.3395,79.5034,77.2003,77.5446,81.2411,78.8578,72.9061,...,79.7689,84.1406,73.9235,83.3767,85.5794,81.8334,81.9578,84.0807,84.2372,82.5103
2026,76.5049,81.7024,75.6975,81.4850,79.6552,77.4142,77.6916,81.4559,79.0327,73.1184,...,80.1179,84.2766,73.9943,83.6144,85.8317,82.0596,82.0715,84.2258,84.4095,82.7872
2027,76.2838,81.6966,75.7784,81.6155,79.8052,77.6268,77.8386,81.6791,79.2107,73.3302,...,80.4606,84.5491,74.0650,83.8521,86.0840,82.2326,82.1894,84.4099,84.5906,83.0618
2028,76.0628,81.6909,75.8517,81.7299,79.9531,77.8348,77.9856,81.9092,79.3904,73.5431,...,80.7969,84.7389,74.1357,84.0897,86.3363,82.4429,82.3037,84.5760,84.7631,83.3343
2029,75.8417,81.6851,75.9178,81.8276,80.0986,78.0446,78.1326,82.1449,79.5711,73.7585,...,81.1269,84.9788,74.2064,84.3274,86.5886,82.6270,82.4222,84.7593,84.9444,83.6046
2030,75.6206,81.6793,75.9782,81.9086,80.2418,78.2528,78.2795,82.3847,79.7517,73.9776,...,81.4505,85.1884,74.2772,84.5650,86.8409,82.8295,82.5371,84.9596,85.1172,83.8730


In [33]:
resultados.to_csv('resultados_EV_10.csv')
https://github.com/grupohenryds03/esperanza_vida/tree/main/datasets # Agregar